In [1]:
%matplotlib inline
from ggplot import *
import matplotlib as mpl
import seaborn as sns
mpl.rcParams['figure.figsize'] = (15, 6)
mpl.style.use('ggplot')
import pandas as pd
import numpy
import subprocess
import glob
import re
import os, sys
from collections import defaultdict, Counter, OrderedDict
from Bio import SeqIO, SeqRecord, Seq
from tRNA_position import *
pd.set_option('display.max_colwidth',10000)
pd.set_option('display.width', 10000)
pd.set_option('display.max_columns', 10000)
pd.set_option('display.max_rows',1000)
isotypes = ['Ala', 'Arg', 'Asn', 'Asp', 'Cys', 'Gln', 'Glu', 'Gly', 'His', 'Ile', 'iMet', 'Leu', 'Lys', 'Met', 'Phe', 'Pro', 'Ser', 'Thr', 'Trp', 'Tyr', 'Val']

# Introduction

A global view of identity elements versus biological features would be a powerful tool for predicting tRNA function using primary sequence. To do this, I'll need to align eukaryotic tRNAs to our established models and annotate positions based on universal numbering. Then, I'll extract sequence information such as position, clade, or variable arm length.

# Process tRNAs
## Species information

In [2]:
species_table = pd.read_table('genomes-051017.tsv', header=None, names=['species', 'longname', 'domain', 'clade', 'taxid'])
species_table.head()

,species,longname,domain,clade,taxid
0,ashbGoss_ATCC10895,Eremothecium gossypii ATCC 10895,eukaryota,Fungi,33169.0
1,aspeFumi_AF293,Aspergillus fumigatus Af293,eukaryota,Fungi,746128.0
2,aspeNidu_FGSC_A4,Aspergillus nidulans FGSC A4,eukaryota,Fungi,162425.0
3,aspeOryz_RIB40,Aspergillus oryzae RIB40,eukaryota,Fungi,5062.0
4,botrCine_B05_10,Botrytis cinerea B05.10,eukaryota,Fungi,40559.0


## Align tRNAs to Sprinzl numbering model

We have a "quality set" of tRNAs, which include tRNAs from previous tRNAscan-SE runs and newer runs. We also want to keep track of high-scoring, isotype mismatched tRNAs.

In [ ]:
species = sorted(glob.glob("out/*-detailed.out"))
species = [sp[4:-19] for sp in species]
seqs = []
for sp in species:  
  sys.stdout.write('processing ' + sp + '...')
  sys.stdout.flush()
  
  tRNA_file = 'tRNAs/' + sp + '-tRNAs.fa'
  tscanout_file = 'out/{}-tRNAs-detailed.out'.format(sp)
  # Import annotation from tRNAscan .out files. We want to keep the ones that are "Quality set" or "Unexpected anticodon;First-pass quality filtered".
  approved_tRNAs = []
  for metadata in pd.read_table(tscanout_file, sep="\t", skiprows=3, header=None).iterrows():
    if (metadata[1].iloc[-1] in ["Quality set", "Unexpected anticodon;First-pass quality filtered", "quality set", "unexpected anticodon,first-pass quality filtered", "Isotype mismatch;First-pass quality filtered"]
        or re.match('^IPD:-\d+\.\d+,isotype mismatch,first-pass quality filtered$', metadata[1].iloc[-1])):
      approved_tRNAs.append('{}.trna{}-{}{}'.format(metadata[1][0], metadata[1][1], metadata[1][4], metadata[1][5]))
 
  # Remove introns using sstofa
  subprocess.call('sstofa3 ss/{}-tRNAs.ss "" 1 0 > {}'.format(sp, tRNA_file), shell=True)

  # Parse isotype-specific scores file
  iso_scores = pd.read_table('iso/' + sp + '-tRNAs.iso', header=0)
  iso_scores['best'] = iso_scores.ix[:,2:].idxmax(axis=1)
  iso_scores['score'] = iso_scores.max(axis=1, numeric_only=True)
  iso_scores.index = iso_scores.tRNAscanID.values
  iso_scores = iso_scores[['best', 'score']]
  
  # Parse seqs, filtering out the ones we don't need, and add to proper isotype
  for seq in SeqIO.parse('tRNAs/{}-tRNAs.fa'.format(sp), 'fasta'):
    if seq.id not in approved_tRNAs: continue
    if "pseudogene" in seq.description: continue
    score = float(re.findall('Sc: [\d\.]+', seq.description)[0].split()[-1])
    if score < 55: continue
    trnascanid = re.findall('.+\.trna\d+', seq.id)[0]
    isotype = iso_scores.ix[trnascanid].best
    if isotype == 'SeC' or 'mito' in isotype or 'mt' in trnascanid: continue
    isoscore = iso_scores.ix[trnascanid].score
    seq.id = '{}|{} Iso: {} ({})'.format(sp, seq.description, isoscore, isotype)
    seq.description = ''
    seqs.append(seq)
    
  print('finished')
  sys.stdout.flush()

In [ ]:
fasta_handle = open('euk-tRNAs.fa', 'w')
SeqIO.write(seqs, fasta_handle, 'fasta')
fasta_handle.close()
num_model = '/projects/lowelab/users/blin/tRNAscan/models/domain-specific/euk-num-092016.cm'
subprocess.call('cmalign -g --notrunc -o euk-tRNAs.sto {} euk-tRNAs.fa'.format(num_model), shell=True)

## Create table of tRNA bases by position

This is a giant data frame with one row per tRNA, and with columns for each position, plus tRNA metadata like species and loop lengths.

In [3]:
def position_base(positions, seq):
  for position_index, position in enumerate(positions):
    if position.paired:
      index1, index2 = position.position.split(':')
      index1, index2 = int(index1), int(index2)
      base_pair = "{}:{}".format(seq[index1 - 1].upper(), seq[index2 - 1].upper())
      yield position.sprinzl, base_pair
    else:
      index = int(position.position)
      base = seq[index - 1].upper()
      yield position.sprinzl, base
      
identities = pd.DataFrame()

# get positions
alignment_fhandle = open('euk-tRNAs.sto')
positions = [] # list containing each position in the tRNA

# first, get secondary structure
for line in alignment_fhandle:
  if line[0:12] == '#=GC SS_cons':
    ss = line.strip().split()[-1]
alignment_fhandle.close()

# parse secondary structure into regions and positions
positions = annotate_positions(ss)

# get nucleotide at each position for each tRNA by parsing Stockholm file
alignment_fhandle = open('euk-tRNAs.sto')
trnas = []
skipped = []
for line in alignment_fhandle:
  if line[0] in ["#", '\n', '/']: continue
  species, desc = line.strip().split('|', 1)
  seqname = desc.split()[0]
  seq = desc.split()[-1]
  if any(species_table.species == species):
    row = species_table[species_table.species == species]
  else: 
    skipped.append(species)
    continue
  isotype = re.findall('\.trna\d+-([A-Za-z]+)', seqname)[0][:-3]
  seqname = '{}_{}'.format(species, seqname)
  trna = {'domain': row.domain.values[0], 'clade': row.clade.values[0], 'species': species, 'species_long': row.longname.values[0], 'taxid': row.taxid.values[0], 'seqname': seqname, 'isotype': isotype}
  trna = {**trna, **{sprinzl: base for sprinzl, base in position_base(positions, seq)}}
  trnas.append(trna)

if len(skipped) > 0: print('skipped the following: {}'.format(set(skipped)))
identities = identities.append(trnas, ignore_index=True)
identities.fillna('.', inplace=True)
alignment_fhandle.close()

skipped the following: {'saccCere_VIN13', 'saccCere_W303', 'saccCere_UFMG_A_905', 'saccCere_R008', 'saccCere_P283', 'saccCere_FOSTERSB', 'saccCere_P301', 'saccCere_VL3', 'saccCere_YJSH1', 'saccCere_CEN_PK113_7D', 'saccCere_R103', 'saccCere_SIGMA1278B', 'saccCere_AWRI796'}


### Create single base columns from paired positions

This is mainly for plotting and such.

In [4]:
cols = list(filter(lambda x: ':' in x, identities.columns))
for col in cols:
  pos1, pos2 = col.split(':')
  base1 = [bases.split(':')[0] for bases in identities[col]]
  base2 = [bases.split(':')[1] for bases in identities[col]]
  identities[pos1] = base1
  identities[pos2] = base2

### Shorten D stem for class II tRNAs

Leu and Tyr have a 3 base pair D stem instead of 4. Since we've already copied over the columns into single bases, this is okay. I don't want to change the entries for this position to "-:-" because it would imply a deletion, and I don't want to change it to ".:." because it would break a lot of code. I'll leave it alone for now, until it's necessary to change it.

### Annotate tertiary interactions

We also want to annotate the 2-base version of the base triples. This makes checking for conserved interactions (e.g. an identity element) a bit easier. It's important to remember to consider the 3rd base pair too, though.

In [5]:
identities['8:14:21'] = identities.ix[:, ["8", "14", "21"]].apply(lambda row: ':'.join(row), axis=1)
identities['8:14'] = identities.ix[:, ["8", "14"]].apply(lambda row: ':'.join(row), axis=1)
identities['9:12:23'] = identities.ix[:, ["9", "12", "23"]].apply(lambda row: ':'.join(row), axis=1)
identities['9:23'] = identities.ix[:, ["9", "23"]].apply(lambda row: ':'.join(row), axis=1)
identities['10:25:45'] = identities.ix[:, ["10", "25", "45"]].apply(lambda row: ':'.join(row), axis=1)
identities['10:45'] = identities.ix[:, ["10", "45"]].apply(lambda row: ':'.join(row), axis=1)
identities['13:22:46'] = identities.ix[:, ["13", "22", "46"]].apply(lambda row: ':'.join(row), axis=1)
identities['22:46'] = identities.ix[:, ["22", "46"]].apply(lambda row: ':'.join(row), axis=1)
identities['15:48'] = identities.ix[:, ["15", "48"]].apply(lambda row: ':'.join(row), axis=1)
identities['18:55'] = identities.ix[:, ["18", "55"]].apply(lambda row: ':'.join(row), axis=1)
identities['19:56'] = identities.ix[:, ["19", "56"]].apply(lambda row: ':'.join(row), axis=1)
identities['26:44'] = identities.ix[:, ["26", "44"]].apply(lambda row: ':'.join(row), axis=1)
identities['54:58'] = identities.ix[:, ["54", "58"]].apply(lambda row: ':'.join(row), axis=1)

### Additional sequence information

In [6]:
# Isotype, anticodon, score
seqinfo = []
for line in open('euk-tRNAs.sto'):
  if line[0:4] != "#=GS": continue
  _, desc, _, _, isotype, anticodon, _, _, _, score, _, isoscore, isotype_best = line.strip().split()
  seqname = desc.replace('|', '_', 1)
  seqinfo.append([seqname, isotype_best[1:-1], anticodon[1:-1], float(score), float(isoscore)])
seqinfo = pd.DataFrame(seqinfo, columns=['seqname', 'isotype_best', 'anticodon', 'score', 'isoscore'])
identities = identities.merge(seqinfo, on='seqname')

# iMet may or may not be properly annotated, depending on tRNAscan-SE run
unlifted_imets = identities.ix[(identities.isotype == "Met") & (identities.isotype_best == "iMet"), ].index
identities.ix[unlifted_imets, 'isotype'] = 'iMet'

# GC content
paired_cols = identities.columns[list(map(lambda x: (':' in x), identities.columns))]
identities['GC'] = identities[paired_cols].apply(lambda x: sum((x == "G:C") | (x == "C:G"))/len(paired_cols), axis=1)

### Insertions/deletions/introns

In [7]:
# Intron length
intron_cols = list(filter(lambda x: x[0:3] == '37i', identities.columns))
identities['intron'] = identities[intron_cols].apply(lambda x: sum(x != "."), axis=1)

# Insertions (minus introns at 37/38)
insertion_cols = list(filter(lambda x: bool(re.search('^\d+i', x)) & (x not in intron_cols), identities.columns))
identities['insertions'] = identities[insertion_cols].apply(lambda x: sum(x != '.'), axis=1)

# Deletions at positions that are not the variable arm, and not counting 17/17a/20a/20b
base_cols = list(filter(lambda x: bool(re.match('^\d+$', x)) & (x not in ['74', '75', '76', '17', '17a', '20a', '20b']), identities.columns))
identities['deletions'] = identities[base_cols].apply(lambda x: ''.join(x).count('-'), axis=1)

### Loop sizes

In [8]:
def bounds_to_cols(cols, start, end):
  selected_cols = []
  for col in cols:
    matches = re.findall('\d+', col)
    if len(matches) < 1: continue
    index = int(matches[0])
    if (index >= start and index <= end or col[0:3] == '{}i'.format(start - 1)) and col[0] != 'V':
      selected_cols.append(col)
  return selected_cols

dloop_cols = list(filter(lambda col: ':' not in col, bounds_to_cols(identities.columns, 14, 21)))
identities['D-loop'] = identities[dloop_cols].apply(lambda x: len(x[(x != '.') & (x != '-')]), axis=1)

# Leu, Ser have a 3 bp D stem
dloop_II_cols = list(filter(lambda col: ':' not in col, bounds_to_cols(identities.columns, 13, 22)))
identities.ix[(identities.isotype == 'Leu') | (identities.isotype == 'Ser'), 'D-loop'] = identities[dloop_II_cols].apply(lambda x: len(x[(x != '.') & (x != '-')]), axis=1)

acloop_cols = list(filter(lambda x: not re.match('37i.+', x), bounds_to_cols(identities.columns, 32, 38)))
identities['AC-loop'] = identities[acloop_cols].apply(lambda x: len(x[(x != '.') & (x != '-')]), axis=1)

tpcloop_cols = bounds_to_cols(identities.columns, 54, 60)
identities['TPC-loop'] = identities[tpcloop_cols].apply(lambda x: len(x[(x != '.') & (x != '-')]), axis=1)

varm_cols = list(filter(lambda x: ('V' in x) & (':' not in x), identities.columns))
identities['V-arm'] = identities[varm_cols].apply(lambda x: len(x[(x != '.') & (x != '-') & (x != "-:-")]), axis=1)

### High-quality tRNA set

There was talk about using the high quality tRNAs to look at identity elements. Instead, I will just annotate specific tRNAs as high-quality or not. We can sort out the differences later.

In [9]:
# Import list of quality tRNAs
quality_trnas = [line.strip() for line in open('quality-set.out')]
identities['quality'] = identities.seqname.isin(quality_trnas)

### Restrict tRNAs by species

We may also want to limit the contribution of any single species, similar to how we built the isotype-specific models. The restricted set is a subset of the quality set.

In [10]:
identities.ix[:, 'restrict'] = False
for species in identities.species.unique():
  species_df = identities.ix[identities.species == species, :]
  for isotype in isotypes:
    isotype_indices = identities.ix[(identities.species == species) & (identities.isotype == isotype) & (-identities.restrict), ].index
    quality_indices = identities.ix[(identities.species == species) & (identities.isotype == isotype) & (-identities.restrict) & (identities.quality), ].index
    unique_scores_indices = isotype_indices[-identities.ix[isotype_indices, 'score'].duplicated()]
    unique_quality_indices = quality_indices[-identities.ix[quality_indices, 'score'].duplicated()]
    
    if len(unique_quality_indices) >= 50:
      unique_quality_indices = unique_quality_indices[numpy.argsort(identities.ix[unique_quality_indices, 'score'])][::-1][:50]
      restricted_indices = list(set(isotype_indices) - set(unique_quality_indices))
    elif len(unique_quality_indices) > 0:
      if len(unique_scores_indices) > 50 - len(unique_quality_indices):
        unique_scores_indices = unique_scores_indices[numpy.argsort(identities.ix[unique_scores_indices, 'score'])][::-1][:50 - len(unique_quality_indices)]
      restricted_indices = list(set(isotype_indices) - set(unique_quality_indices) - set(unique_scores_indices))
    else:
      if len(unique_scores_indices) > 50:
        unique_scores_indices = unique_scores_indices[numpy.argsort(identities.ix[unique_scores_indices, 'score'])][::-1][:50]
      restricted_indices = list(set(isotype_indices) - set(unique_scores_indices))
    
    identities.ix[restricted_indices, 'restrict'] = True

The quality set does have some issues, though: it filters out rare codons, when rare is relative to model species. It also performs a hard check for anticodon-isotype model match. Fungi have much more diverged tRNAs and, in many cases, a different isotype scores higher even when it's clear that it's the bona fide tRNA (mostly based on number of tRNAs of that isotype, or codon usage). 

## Export to R

R has superior visualization capabilities.

### Order columns

To make it look pretty.

In [11]:
def position_str_to_int(position):
  if position == "20a": return 20.1
  if position == "20b": return 20.2
  digits = re.findall('\d+', position)
  if len(digits) == 0: return -1
  insert = 0
  if 'i' in position and len(digits) == 2: insert = float(digits[1]) / 1000
  if position[0] == 'V':
    if ':' in position: return int(digits[0]) + 45 - 10 + insert # V11~V17
    else: return int(digits[0]) + 45 + 7 + insert # V1~V5
  if int(digits[0]) >= 46: return int(digits[0]) + 50 + insert # just add an arbitrarily large number to skip v-arm
  return int(digits[0]) + insert

identities = identities[sorted(list(identities.columns), key=position_str_to_int)]

In [12]:
identities.to_csv(path_or_buf='identities.tsv', sep='\t', index_label=False)

In [13]:
identities.head()

,clade,domain,isotype,seqname,species,species_long,taxid,isotype_best,anticodon,score,isoscore,GC,intron,insertions,deletions,D-loop,AC-loop,TPC-loop,V-arm,quality,restrict,1:72,1,1i1,2:71,2,2i1,3:70,3,3i1,3i2,3i3,3i4,3i5,3i6,4:69,4,4i1,4i2,4i3,4i4,4i5,4i6,4i7,4i8,4i9,4i10,4i11,5:68,5,5i1,5i2,5i3,5i4,5i5,5i6,6:67,6,6i1,7:66,7,7i1,7i2,7i3,7i4,7i5,7i6,7i7,7i8,7i9,8,8:14:21,8:14,8i1,8i2,9,9:12:23,9:23,9i1,10:25,10,10:25:45,10:45,10i1,11:24,11,12:23,12,12i1,13:22,13,13:22:46,14,14i1,14i2,14i3,14i4,14i5,14i6,14i7,14i8,14i9,14i10,14i11,14i12,14i13,15,15:48,16,16i1,16i2,16i3,16i4,16i5,16i6,16i7,16i8,16i9,16i10,16i11,16i12,16i13,16i14,16i15,16i16,17,17a,18,18:55,19,19:56,19i1,19i2,19i3,19i4,19i5,19i6,19i7,19i8,19i9,19i10,20,20i1,20i2,20i3,20i4,20a,20b,21,22,22:46,22i1,23,23i1,23i2,23i3,23i4,23i5,23i6,24,24i1,25,25i1,25i2,25i3,25i4,26,26:44,26i1,26i2,26i3,26i4,26i5,26i6,26i7,26i8,26i9,26i10,27:43,27,27i1,27i2,27i3,27i4,28:42,28,28i1,29:41,29,29i1,29i2,30:40,30,30i1,31:39,31,32,33,34,35,35i1,36,37,37i1,37i2,37i3,37i4,37i5,37i6,37i7,37i8,37i9,38,38i1,39,39i1,39i2,40,40i1,40i2,41,41i1,41i2,42,42i1,43,44,44i1,44i2,44i3,44i4,44i5,44i6,44i7,44i8,44i9,44i10,44i11,44i12,44i13,44i14,44i15,44i16,44i17,44i18,44i19,44i20,44i21,44i22,44i23,45,V11:V21,V12:V22,V13:V23,V14:V24,V15:V25,V16:V26,V17:V27,V1,V2,V3,V4,V5,V11,V12,V13,V14,V15,V16,V17,V21,V22,V23,V24,V25,V26,V27,46,47,47i1,47i2,47i3,48,49:65,49,49i1,50:64,50,50i1,50i2,50i3,50i4,50i5,50i6,50i7,51:63,51,51i1,51i2,51i3,52:62,52,52i1,53:61,53,53i1,54,54:58,54i1,54i2,54i3,54i4,55,55i1,55i2,55i3,55i4,55i5,56,56i1,56i2,56i3,56i4,57,57i1,58,58i1,58i2,59,59i1,60,60i1,60i2,60i3,60i4,60i5,60i6,60i7,60i8,60i9,60i10,60i11,61,61i1,62,63,64,64i1,64i2,65,65i1,65i2,65i3,65i4,65i5,65i6,65i7,65i8,65i9,66,66i1,67,67i1,68,68i1,68i2,69,69i1,70,70i1,70i2,70i3,70i4,70i5,70i6,70i7,70i8,70i9,71,71i1,71i2,72,73,74,75,76
0,Insecta,eukaryota,Ser,aedAeg1_NW_001809801.1.trna3-SerAGA,aedAeg1,Aedes aegypti (yellow fever mosquito),.,Arg,AGA,63.6,62.3,0.390244,0,2,1,10,7,8,0,False,False,C:G,C,.,C:A,C,.,U:A,U,.,.,.,.,.,.,C:G,C,.,.,.,.,.,.,.,.,.,.,.,U:A,U,.,.,.,.,.,.,C:G,C,.,A:U,A,.,.,.,.,.,.,.,.,.,U,U:A:A,U:A,.,.,G,G:G:C,G:C,.,G:C,G,G:C:-,G:-,.,C:G,C,G:C,G,.,U:A,U,U:A:G,A,.,.,.,.,.,.,.,.,.,.,.,.,.,G,G:C,U,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,-,-,G,G:U,G,G:C,.,.,.,.,.,.,.,.,.,.,U,.,.,.,.,U,-,A,A,A:G,.,C,.,.,.,.,.,.,G,.,C,.,.,.,.,G,G:A,.,.,.,.,.,.,.,.,.,.,C:G,C,.,.,.,.,C:G,C,.,C:G,C,.,.,C:G,C,A,A:U,A,C,U,A,G,.,A,G,.,.,.,.,.,.,.,.,.,A,.,U,.,C,G,.,.,G,.,.,G,.,G,A,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,-,-:-,-:-,-:-,-:-,-:-,-:-,-:-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,G,U,.,.,.,C,G:C,G,.,U:A,U,.,.,.,.,.,.,.,G:C,G,.,.,.,A:U,A,.,G:C,G,.,U,U:A,.,.,.,.,U,.,.,.,.,.,C,.,.,.,.,G,.,A,.,.,U,.,U,.,.,.,.,.,.,.,.,.,.,.,C,.,U,C,A,.,.,C,.,.,.,.,.,.,.,.,.,U,.,G,.,A,.,.,G,.,A,.,.,.,.,.,.,.,.,.,A,.,.,G,A,-,-,-
1,Insecta,eukaryota,Ser,aedAeg1_NW_001809801.1.trna4-SerAGA,aedAeg1,Aedes aegypti (yellow fever mosquito),.,Arg,AGA,63.6,62.3,0.390244,0,2,1,10,7,8,0,False,True,C:G,C,.,C:A,C,.,U:A,U,.,.,.,.,.,.,C:G,C,.,.,.,.,.,.,.,.,.,.,.,U:A,U,.,.,.,.,.,.,C:G,C,.,A:U,A,.,.,.,.,.,.,.,.,.,U,U:A:A,U:A,.,.,G,G:G:C,G:C,.,G:C,G,G:C:-,G:-,.,C:G,C,G:C,G,.,U:A,U,U:A:G,A,.,.,.,.,.,.,.,.,.,.,.,.,.,G,G:C,U,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,-,-,G,G:U,G,G:C,.,.,.,.,.,.,.,.,.,.,U,.,.,.,.,U,-,A,A,A:G,.,C,.,.,.,.,.,.,G,.,C,.,.,.,.,G,G:A,.,.,.,.,.,.,.,.,.,.,C:G,C,.,.,.,.,C:G,C,.,C:G,C,.,.,C:G,C,A,A:U,A,C,U,A,G,.,A,G,.,.,.,.,.,.,.,.,.,A,.,U,.,C,G,.,.,G,.,.,G,.,G,A,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,-,-:-,-:-,-:-,-:-,-:-,-:-,-:-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,G,U,.,.,.,C,G:C,G,.,U:A,U,.,.,.,.,.,.,.,G:C,G,.,.,.,A:U,A,.,G:C,G,.,U,U:A,.,.,.,.,U,.,.,.,.,.,C,.,.,.,.,G,.,A,.,.,U,.,U,.,.,.,.,.,.,.,.,.,.,.,C,.,U,C,A,.,.,C,.,.,.,.,.,.,.,.,.,U,.,G,.,A,.,.,G,.,A,.,.,.,.,.,.,.,.,.,A,.,.,G,A,-,-,-
2,Insecta,eukaryota,Ser,aedAeg1_NW_001809801.1.trna5-SerAGA,aedAeg1,Aedes aegypti (yellow fever mosquito),.,Arg,AGA,63.6,62.3,0.390244,0,2,1,10,7,8,0,False,True,C:G,C,.,C:A,C,.,U

# Resolve consensus positions

There are some discrepencies between the covariance model, the Sprinzl model, and reality. For instance, is position 17 really a gap position for most tRNAs? If so, why is it considered a position? We know that the D loop interacts with the TPC loop, so the general position 17 area should be a key identity element or structural feature contributing to tRNA activity. What should we call insertions at 17 if it's not a real position? These conflicts need to be resolved into a consensus version.

In [ ]:
def bounds_to_cols(cols, start, end):
  selected_cols = []
  for col in cols:
    matches = re.findall('\d+', col)
    if len(matches) < 1: continue
    index = int(matches[0])
    if (index >= start and index <= end or col[0:3] == '{}i'.format(start - 1)) and col[0] != 'V':
      selected_cols.append(col)
  return selected_cols

def position_histogram(identities, cols):
  df = pd.melt(identities[cols + ['seqname']], id_vars=['seqname'])
  df = df[(df.value != ".") & (df.value != "-")]
  counts = pd.Series(Counter([x for x in df.variable]), index=cols)
  counts = counts[counts > 10]
  counts = pd.DataFrame({"Position": counts.index, 'tRNAs': counts})
  return counts.plot(kind='bar')

### D-loop positions

In [ ]:
dloop_cols = bounds_to_cols(identities.columns, 14, 21)
position_histogram(identities, dloop_cols)

With the model curated using --hand, the D loop now contains 17a, 20a, and 20b. The alignment seems to have been fixed, as we don't have a bump at 20i5 (or even 20i5-20i7) anymore.

#### 17A

What isotypes have position 17?

In [ ]:
df = pd.melt(identities[['isotype', '17']], id_vars=['isotype'])
df = df[(df.value != ".") & (df.value != "-")]
df = pd.Series(Counter([x for x in df.isotype]))
df.plot(kind='bar')

In [ ]:
df = pd.melt(identities[['isotype', '17a']], id_vars=['isotype'])
df = df[(df.value != ".") & (df.value != "-")]
df = pd.Series(Counter([x for x in df.isotype]))
df.plot(kind='bar')

Based on Sprinzl alignments, there are 19 tRNAs out of 2249 with a base at position 17A. It also doesn't look like we can hardcode position 17 for just one or two isotypes - it's universal enough. Based on this data, 17A is pretty rare, and even position 17 is only there 30% of the time (based on our data). Position 17A could actually be considered as an insertion.

#### 20a/20b

** This section is somewhat deprecated as the alignment has been fixed. ** The following analysis has been modified to double check whether the new 20a/20b alignment is sound.

Also based on Sprinzl alignments, no tRNAs have only 20b and not 20a. So whenever there is 20b, 20a is sure to be there. Let's take a look at the distribution out of 2249 tRNA genes:

In [ ]:
sprinzl_20ab = pd.read_table('sprinzl-20ab.txt', header=None, names=['20a', '20b', 'Isotype'])
sprinzl_20ab = sprinzl_20ab.groupby(["Isotype"]).aggregate(lambda x: len(x[x != '-']))
sprinzl_20ab.plot(kind="bar")

This complicates things. Ideally we'd like to see all of the tRNAs with 20b to belong to one or two isotypes. Then we could manually check the most common insertion positions for those isotypes and adjust it. Instead, we'll just have to check the insertion positions for all isotypes and attempt to come up with some universal rules.

In [ ]:
sprinzl_20ab = pd.read_table('sprinzl-20ab.txt', header=None, names=['20a', '20b', 'Isotype'])
sprinzl_20ab = sprinzl_20ab.groupby(["20a", "20b"]).aggregate(len)
sprinzl_20ab.plot(kind="bar")

This plot tells us a few things.

First, 20a is typically a T, but there are a few ACG 20as. To be expected, as 20ab are typically modified to dihydrouridine (D).

20b is typically also a T, but is most commonly paired with C20a. There are a few of the others, too. [Nissan and Perona (2000)](https://www.ncbi.nlm.nih.gov/pubmed/11105758) corroborate this by showing a successful replacement of  yeast function serine U20a/G20b with C20a/A20b _in vitro_. This complicates things. 

I hesitate to use a rule like "First base is 20a, second base is 20b, third and up is 20i". There is high potential for misannotation of an insertion as 20ab or vice versa. We can agree on the following:
- single base insertion: 20a
- two base insertion: 20a, 20b

For three and up, we could potentially leave it unannotated. Let's take a look at the relative insertion lengths, by isotype:

In [ ]:
insert_df = identities[['isotype'] + ['20a', '20b', '20i1', '20i2', '20i3', '20i4']]
insertionab = insert_df.ix[:, 1:].apply(lambda x: sum((x != '.') & (x != '-')), axis=1)
insert_df = insert_df.assign(insertionab = insertionab)

In [ ]:
print(insert_df.groupby('insertionab').aggregate({"isotype": len}))
plot = sns.violinplot(data=insert_df, x='isotype', y='insertionab', scale='width', cut=0)
plot.set(ylim=(-1, 5))

Half of the tRNAs have no 20a/20b, and the other half does. For 10 isotypes, all tRNAs are 2 insertions and below. For the other 10, it varies. Exactly how many tRNAs are we working with here? Is there a correlation betweeen number of tRNAs and number of tRNAs with insertions > 2?

In [ ]:
df = insert_df.ix[insert_df.insertionab > 2, ['isotype', 'insertionab']].groupby('isotype').aggregate(len)
print("No. tRNAs with > 2 insertions: {}".format(sum(df.insertionab)))
df = df.merge(identities[['isotype', 'seqname']].groupby('isotype').aggregate(len), how='right', right_index=True, left_index=True)
df = df.fillna(0)
df.columns = ['insertionab_tRNAs', 'tRNAs']

# scatter plot
plot = df.plot(kind='scatter', x='insertionab_tRNAs', y='tRNAs')
for isotype, num_tRNAs in df.iterrows(): plot.annotate(isotype, num_tRNAs, xytext=(5, 0), textcoords='offset points', family='sans-serif')

# add trendline
beta = numpy.polyfit(x=df.insertionab_tRNAs, y=df.tRNAs, deg=1)
trendline = pd.DataFrame({'x': list(df.insertionab_tRNAs) + [-20, 120], 'trendline': numpy.poly1d(beta)(list(df.insertionab_tRNAs) + [-20, 120])})
trendline.plot(ax=plot, x='x', y='trendline', kind='line', xlim=[-3, 10], ylim=[0, 2500])

plot.set_xlabel('No. tRNAs with >2 insertions at position 20')
plot.set_ylabel('No. tRNAs')

There's only 38 tRNAs with a 3 base or larger insertion. Here they are below. They're mostly leucine tRNAs.

In [ ]:
df = insert_df[insert_df.insertionab > 2]
df["seq"] = df[['20a', '20b', '20i1', '20i2', '20i3', '20i4']].apply(lambda x: ''.join(x[(x != '.') & (x != '-')]), axis=1)
df.sort_values('insertionab')[['isotype', 'seq']]

### Variable stem-loop plus spacer nucleotides

** This section is somewhat deprecated as the alignment has been fixed. ** The following analysis has been modified to double check whether the position 47 and V-arm alignment is sound.

In [ ]:
varm_cols = list(filter(lambda x: ('V' in x) & (':' not in x), identities.columns))
varm_cols += bounds_to_cols(identities.columns, 44, 48)
position_histogram(identities, varm_cols)

This data correlates well with the Sprinzl data. Most variable stems are missing at least V15:V25, V16:V26, and V17:V27, and most variable loops are 3 nucleotides long.

Here is the same plot, but split into Ser, Leu, and Tyr, versus everything else.

In [ ]:
df = pd.melt(identities[varm_cols + ['seqname', 'isotype']], id_vars=['seqname', 'isotype'])
df = df[(df.value != ".") & (df.value != "-")]
counts = pd.Series(Counter([x for x in df[(df.isotype == "Tyr") | (df.isotype == "Leu") | (df.isotype == "Ser")].variable]), index=varm_cols)
counts = pd.DataFrame({"Position": counts.index, 'tRNAs': counts})
counts2 = pd.Series(Counter([x for x in df[(df.isotype != "Tyr") | (df.isotype != "Leu") | (df.isotype != "Ser")].variable]), index=varm_cols)
counts = pd.merge(counts, pd.DataFrame({"Position": counts2.index, 'tRNAs': counts2}), on=['Position'])
counts.columns = ['Position', 'Class II', 'Class I']
counts.index = counts.Position
counts.plot(kind='bar')

We need to apply a similar analysis for 47 as we did for 20ab. What does the distribution of 47 + insert length look like?

In [ ]:
insert_df = identities[['isotype', '47', '47i1']]
insert_df.head()
insertion47 = 2 - sum([(insert_df.ix[:, 1] == "."), (insert_df.ix[:, 2] == "."), (insert_df.ix[:, 1] == "-"), (insert_df.ix[:, 2] == "-")])
insert_df = insert_df.assign(insertion47 = insertion47)
print(insert_df.groupby('insertion47').aggregate({"isotype": len}))

Of the 94197 tRNAs with 1 nucleotide at position 47 + 47ix, which position does it appear on? Is there a typical position 47 nucleotide?

In [ ]:
df = pd.melt(insert_df.ix[insert_df.insertion47 == 1, 0:3], id_vars='isotype')
df = df[(df.value != "-") & (df.value != '.')]
df[['variable', 'value']].groupby(['variable', 'value']).aggregate(len).unstack().plot(kind='bar')
df.ix[df.variable != "47", :]

There are 0 cases where a single base is not aligned to position 47 properly.